In [2]:
!pip install --quiet gradio
!pip install shap
!git clone https://github.com/ultralytics/yolov5
!git clone https://github.com/biplav-s/image-mask-estimate

     |████████████████████████████████| 1.1MB 17.2MB/s 
     |████████████████████████████████| 215kB 58.9MB/s 
     |████████████████████████████████| 2.6MB 49.9MB/s 
     |████████████████████████████████| 71kB 10.1MB/s 
     |████████████████████████████████| 962kB 47.2MB/s 
     |████████████████████████████████| 358kB 17.9MB/s 
  Created wheel for shap: filename=shap-0.38.1-cp36-cp36m-linux_x86_64.whl size=489408 sha256=2a1c07cc0397f7f878b5a1da199049f5362e4c59b5817c7a6848c2ac2feadfa4
  Stored in directory: /root/.cache/pip/wheels/a8/fb/e4/88012be41842b9be62ae18d82d1b1e880daf8539d1fef1fa00
Successfully built shap
Cloning into 'yolov5'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 4558 (delta 34), reused 38 (delta 19), pack-reused 4491
Receiving objects: 100% (4558/4558), 7.72 MiB | 36.10 MiB/s, done.
Resolving deltas: 100% (3096/3096), done.
Cloning into 'image-mask-estimate'...

In [62]:
!pip install lime

     |████████████████████████████████| 276kB 14.3MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-cp36-none-any.whl size=283846 sha256=83ee0e8c660abe823fac37f0139049069a3e32cd65d73afcc93bdb4e98603207
  Stored in directory: /root/.cache/pip/wheels/4c/4f/a5/0bc765457bd41378bf3ce8d17d7495369d6e7ca3b712c60c89
Successfully built lime


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
%cp /content/gdrive/My\ Drive/best.pt /content

In [63]:
from lime import lime_image
from skimage.segmentation import mark_boundaries

In [7]:
import gradio as gr
import numpy as np
import requests
import random
from glob import glob
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import shap
import torchvision
import cv2
from utils.general import non_max_suppression, box_iou
device = torch.device('cuda:0')

In [17]:
%matplotlib inline

In [8]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

In [12]:
model = torch.hub.load('ultralytics/yolov5','custom', path_or_model='/content/best.pt')
detector1 = model.autoshape() 

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Focus                     [3, 32, 3]                    
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1    156928  models.common.C3                        [128, 128, 3]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  1    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672

Adding autoShape... 
autoShape already enabled, skipping... 


In [109]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [110]:
image_in = gr.inputs.Image()
image_ou = gr.outputs.Image()
text_ou = gr.outputs.Textbox()

In [111]:
def classify_image(inp):
    print(inp.shape)
    pred = detector1(inp)
    pred.save()
    mask=0
    non_mask =0
    for i,box in enumerate(pred.xyxy[0]):
      if int(box[5])==1:
        non_mask = non_mask+1
      else :
        mask = mask+1

    img = cv2.imread('/content/results0.jpg',cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if (mask+non_mask)!=0:
      res_string = f'Prediction: {mask} people wearing mask and {non_mask} people are not wearing mask; Proportion: {mask/(mask+non_mask):2.3f}'
    else:
      res_string = 'Prediction: Not identify any People'

    return [img, res_string]

In [120]:
def Yolo_sum(images):
    pred = []
    for image in images:
      sum=0
      logits = detector1(image)
      result = np.array(logits.xyxy[0][:,4])
      for i in result:
        sum+=i
      sum = sum/len(result) if len(result) else 0 
      pred.append([sum,1-sum])
    return np.array(pred)

In [121]:
#lime explainer
explainer = lime_image.LimeImageExplainer()
def lime_result(img): 
  
    explanation = explainer.explain_instance(np.array(img),Yolo_sum , top_labels=5, hide_color=0, num_samples=100)
    
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=10, hide_rest=False)
    img_boundry1 = mark_boundaries(temp/255.0, mask)
    
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=False)
    img_boundry2 = mark_boundaries(temp/255.0, mask)
    
    f, axarr = plt.subplots(1,2)
    axarr[0].imshow(img_boundry1)
    axarr[1].imshow(img_boundry2)
    plt.savefig(f'result.jpg')

    img = cv2.imread('/content/result.jpg',cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    return [ img, 'Explaining result with segmentation process']



In [ ]:
gr.Interface(
    [classify_image,lime_result], 
    image_in, 
    [image_ou, text_ou],
    title = "Mask Detection & Explaination",
    description = "In this model we use Yolov5 fine tuned on Mask Detection dataset. For explaining our we use LIME - 'Local Interpretable Model-Agnostic' using segmentaion process ",
    capture_session=True).launch(debug="True");